##### Copyright 2020 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# 计算图和 tf.function 简介

<table class="tfo-notebook-buttons" align="left">
  <td><a target="_blank" href="https://www.tensorflow.org/guide/intro_to_graphs"><img src="https://www.tensorflow.org/images/tf_logo_32px.png">View 在 TensorFlow.org 上查看</a></td>
  <td><a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs/blob/master/site/en/guide/intro_to_graphs.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png">在 Google Colab 中运行 </a></td>
  <td><a target="_blank" href="https://github.com/tensorflow/docs/blob/master/site/en/guide/intro_to_graphs.ipynb">     <img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png">     在 GitHub 上查看源代码</a></td>
  <td><a href="https://storage.googleapis.com/tensorflow_docs/docs/site/en/guide/intro_to_graphs.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png">下载笔记本</a></td>
</table>

## 概述

本指南深入探究 TensorFlow 和 Keras 以演示 TensorFlow 的工作原理。如果您想立即开始使用 Keras，请查看 [Keras 指南合集](https://www.tensorflow.org/guide/keras/)。

在本指南中，您将了解 TensorFlow 如何让您对代码进行简单的更改来获取计算图、计算图的存储和表示方式以及如何使用它们来加速您的模型。

注：对于那些只熟悉 TensorFlow 1.x 的用户来说，本指南演示了迥然不同的计算图视图。

**This is a big-picture overview that covers how `tf.function` allows you to switch from eager execution to graph execution.** For a more complete specification of `tf.function`, go to the <a href="./function.ipynb" data-md-type="link">Better performance with `tf.function`</a> guide.


### 什么是计算图？

在前三篇指南中，您以 **Eager** 模式运行了 TensorFlow。这意味着 TensorFlow 运算由 Python 逐个执行，然后将结果返回给 Python。

虽然 Eager Execution 具有多个独特的优势，但计算图执行在 Python 外部实现了可移植性，并且往往提供更出色的性能。**计算图执行**意味着张量计算作为 *TensorFlow 计算图*执行，后者有时被称为 `tf.Graph` 或简称为“计算图”。

**计算图是包含一组 `tf.Operation` 对象（表示计算单元）和 `tf.Tensor` 对象（表示在运算之间流动的数据单元）的数据结构。**计算图在 `tf.Graph` 上下文中定义。由于这些计算图是数据结构，无需原始 Python 代码即可保存、运行和恢复它们。

This is what a TensorFlow graph representing a two-layer neural network looks like when visualized in TensorBoard:

<img src="https://github.com/tensorflow/docs/blob/master/site/en/guide/images/intro_to_graphs/two-layer-network.png?raw=1" alt="一个简单的 TensorFlow 计算图">

### 计算图的优点

With a graph, you have a great deal of flexibility.  You can use your TensorFlow graph in environments that don't have a Python interpreter, like mobile applications, embedded devices, and backend servers. TensorFlow uses graphs as the format for [saved models](./saved_model.ipynb) when it exports them from Python.

计算图的优化也十分轻松，允许编译器进行如下转换：

- 通过在计算中折叠常量节点来静态推断张量的值*（“常量折叠”）*。
- 分离独立的计算子部分，并在线程或设备之间进行拆分。
- 通过消除通用子表达式来简化算术运算。


有一个完整的优化系统 [Grappler](./graph_optimization.ipynb) 来执行这种加速和其他加速。

简而言之，计算图极为有用，它可以使 TensorFlow **快速**运行、**并行**运行以及**在多个设备上**高效运行。

但是，为了方便起见，您仍然需要在 Python 中定义我们的机器学习模型（或其他计算），然后在需要时自动构造计算图。

## 设置

Import some necessary libraries:

In [ ]:
import tensorflow as tf
import timeit
from datetime import datetime

## 利用计算图

您可以使用 `tf.function` 在 TensorFlow 中创建和运行计算图，要么作为直接调用，要么作为装饰器。`tf.function` 将一个常规函数作为输入并返回一个 `Function`。<strong data-md-type="double_emphasis">`Function` 是一个 Python 可调用对象，它通过 Python 函数构建 TensorFlow 计算图。您可以按照与其 Python 等价函数相同的方式使用 `Function`。</strong>


In [ ]:
# Define a Python function.
def a_regular_function(x, y, b):
  x = tf.matmul(x, y)
  x = x + b
  return x

# `a_function_that_uses_a_graph` is a TensorFlow `Function`.
a_function_that_uses_a_graph = tf.function(a_regular_function)

# Make some tensors.
x1 = tf.constant([[1.0, 2.0]])
y1 = tf.constant([[2.0], [3.0]])
b1 = tf.constant(4.0)

orig_value = a_regular_function(x1, y1, b1).numpy()
# Call a `Function` like a Python function.
tf_function_value = a_function_that_uses_a_graph(x1, y1, b1).numpy()
assert(orig_value == tf_function_value)

On the outside, a `Function` looks like a regular function you write using TensorFlow operations. [Underneath](https://github.com/tensorflow/tensorflow/blob/master/tensorflow/python/eager/def_function.py), however, it is *very different*. A `Function` **encapsulates several `tf.Graph`s behind one API** (learn more in the _Polymorphism_ section). That is how a `Function` is able to give you the benefits of graph execution, like speed and deployability (refer to _The benefits of graphs_ above).

`tf.function` 适用于一个函数*及其调用的所有其他函数*：

In [ ]:
def inner_function(x, y, b):
  x = tf.matmul(x, y)
  x = x + b
  return x

# Use the decorator to make `outer_function` a `Function`.
@tf.function
def outer_function(x):
  y = tf.constant([[2.0], [3.0]])
  b = tf.constant(4.0)

  return inner_function(x, y, b)

# Note that the callable will create a graph that
# includes `inner_function` as well as `outer_function`.
outer_function(tf.constant([[1.0, 2.0]])).numpy()

如果您使用过 TensorFlow 1.x，会发现根本不需要定义 `Placeholder` 或 `tf.Sesssion`。

### 将 Python 函数转换为计算图

您使用 TensorFlow 编写的任何函数都将包含内置 TF 运算和 Python 逻辑的混合，例如 `if-then` 子句、循环、`break`、`return`、`continue` 等。虽然 TensorFlow 运算很容易被 `tf.Graph` 捕获，但特定于 Python 的逻辑需要经过额外的步骤才能成为计算图的一部分。`tf.function` 使用称为 AutoGraph (`tf.autograph`) 的库将 Python 代码转换为计算图生成代码。


In [ ]:
def simple_relu(x):
  if tf.greater(x, 0):
    return x
  else:
    return 0

# `tf_simple_relu` is a TensorFlow `Function` that wraps `simple_relu`.
tf_simple_relu = tf.function(simple_relu)

print("First branch, with graph:", tf_simple_relu(tf.constant(1)).numpy())
print("Second branch, with graph:", tf_simple_relu(tf.constant(-1)).numpy())

虽然您不太可能需要直接查看计算图，但您可以检查输出以验证确切的结果。这些结果都不太容易阅读，因此不需要看得太仔细！

In [ ]:
# This is the graph-generating output of AutoGraph.
print(tf.autograph.to_code(simple_relu))

In [ ]:
# This is the graph itself.
print(tf_simple_relu.get_concrete_function(tf.constant(1)).graph.as_graph_def())

Most of the time, `tf.function` will work without  special considerations. However, there are some caveats, and the [`tf.function` guide](./function.ipynb) can help here, as well as the [complete AutoGraph reference](https://github.com/tensorflow/tensorflow/blob/master/tensorflow/python/autograph/g3doc/reference/index.md).

### 多态性：一个 `Function`，多个计算图

`tf.Graph` 专门用于特定类型的输入（例如，具有特定 [`dtype`](https://www.tensorflow.org/api_docs/python/tf/dtypes/DType) 的张量或具有相同 [`id()`](https://docs.python.org/3/library/functions.html#id%5D)) 的对象）。

Each time you invoke a `Function` with a set of arguments that can't be handled by any of its existing graphs (such as arguments with new `dtypes` or incompatible shapes), `Function` creates a new `tf.Graph` specialized to those new arguments. The type specification of a `tf.Graph`'s inputs is known as its **input signature** or just a **signature**. For more information regarding when a new `tf.Graph` is generated and how that can be controlled, go to the *Rules of tracing* section of the [Better performance with `tf.function`](./function.ipynb) guide.

`Function` 在 `ConcreteFunction` 中存储与该签名对应的 `tf.Graph`。<strong data-md-type="double_emphasis">`ConcreteFunction` 是围绕 `tf.Graph` 的封装容器</strong>。


In [ ]:
@tf.function
def my_relu(x):
  return tf.maximum(0., x)

# `my_relu` creates new graphs as it observes more signatures.
print(my_relu(tf.constant(5.5)))
print(my_relu([1, -1]))
print(my_relu(tf.constant([3., -3.])))

如果已经使用该签名调用了 `Function`，则该 `Function` 不会创建新的 `tf.Graph`。

In [ ]:
# These two calls do *not* create new graphs.
print(my_relu(tf.constant(-2.5))) # Signature matches `tf.constant(5.5)`.
print(my_relu(tf.constant([-1., 1.]))) # Signature matches `tf.constant([3., -3.])`.

Because it's backed by multiple graphs, a `Function` is **polymorphic**. That enables it to support more input types than a single `tf.Graph` could represent, and to optimize each `tf.Graph` for better performance.

In [ ]:
# There are three `ConcreteFunction`s (one for each graph) in `my_relu`.
# The `ConcreteFunction` also knows the return type and shape!
print(my_relu.pretty_printed_concrete_signatures())

## 使用 `tf.function`

到目前为止，您已经学习了如何使用 `tf.function` 作为装饰器或包装容器将 Python 函数简单地转换为计算图。但在实践中，让 `tf.function` 正常工作可能相当棘手！在下面的部分中，您将了解如何使用 `tf.function` 使代码按预期工作。

### 计算图执行与 Eager Execution

`Function` 函数中的代码既能以 Eager 模式执行，也可以作为计算图执行。默认情况下，`Function` 将其代码作为计算图执行：


In [ ]:
@tf.function
def get_MSE(y_true, y_pred):
  sq_diff = tf.pow(y_true - y_pred, 2)
  return tf.reduce_mean(sq_diff)

In [ ]:
y_true = tf.random.uniform([5], maxval=10, dtype=tf.int32)
y_pred = tf.random.uniform([5], maxval=10, dtype=tf.int32)
print(y_true)
print(y_pred)

In [ ]:
get_MSE(y_true, y_pred)

To verify that your `Function`'s graph is doing the same computation as its equivalent Python function, you can make it execute eagerly with `tf.config.run_functions_eagerly(True)`. This is a switch that **turns off `Function`'s ability to create and run graphs**, instead of executing the code normally.

In [ ]:
tf.config.run_functions_eagerly(True)

In [ ]:
get_MSE(y_true, y_pred)

In [ ]:
# Don't forget to set it back when you are done.
tf.config.run_functions_eagerly(False)

但是，`Function` 在计算图执行和 Eager Execution 下的行为可能有所不同。Python [`print`](https://docs.python.org/3/library/functions.html#print) 函数是这两种模式不同之处的一个示例。我们看看当您将 `print` 语句插入到您的函数并重复调用它时会发生什么。

In [ ]:
@tf.function
def get_MSE(y_true, y_pred):
  print("Calculating MSE!")
  sq_diff = tf.pow(y_true - y_pred, 2)
  return tf.reduce_mean(sq_diff)

观察打印的内容：

In [ ]:
error = get_MSE(y_true, y_pred)
error = get_MSE(y_true, y_pred)
error = get_MSE(y_true, y_pred)

输出很令人惊讶？尽管 **`get_MSE` 被调用了 *3* 次，但它只打印了一次。**

To explain, the `print` statement is executed when `Function` runs the original code in order to create the graph in a process known as "tracing" (refer to the *Tracing* section of the [`tf.function` guide](./function.ipynb). <strong data-md-type="double_emphasis">Tracing captures the TensorFlow operations into a graph, and `print` is not captured in the graph.</strong>  That graph is then executed for all three calls **without ever running the Python code again**.

作为健全性检查，我们关闭计算图执行来比较：

In [ ]:
# Now, globally set everything to run eagerly to force eager execution.
tf.config.run_functions_eagerly(True)

In [ ]:
# Observe what is printed below.
error = get_MSE(y_true, y_pred)
error = get_MSE(y_true, y_pred)
error = get_MSE(y_true, y_pred)

In [ ]:
tf.config.run_functions_eagerly(False)

`print` is a *Python side effect*, and there are other differences that you should be aware of when converting a function into a `Function`. Learn more in the _Limitations_ section of the [Better performance with `tf.function`](./function.ipynb) guide.

注：如果您想同时在 Eager Execution 和计算图执行中打印值，请改用 `tf.print`。

### 非严格执行

<a id="non-strict"></a>

计算图执行仅执行产生可观察效果所需的运算，这包括：

- 函数的返回值
- 已记录的著名副作用，例如：
    - 输入/输出运算，如 `tf.print`
    - 调试运算，如 `tf.debugging` 中的断言函数
    - `tf.Variable` 的突变

这种行为通常称为“非严格执行”，与 Eager Execution 不同，后者会分步执行所有程序运算，无论是否需要。

特别是，运行时错误检查不计为可观察效果。如果一个运算因为不必要而被跳过，它不会引发任何运行时错误。

在下面的示例中，计算图执行期间跳过了“不必要的”运算 `tf.gather`，因此不会像在 Eager Execution 中那样引发运行时错误 `InvalidArgumentError`。切勿依赖执行计算图时引发的错误。

In [ ]:
def unused_return_eager(x):
  # Get index 1 will fail when `len(x) == 1`
  tf.gather(x, [1]) # unused 
  return x

try:
  print(unused_return_eager(tf.constant([0.0])))
except tf.errors.InvalidArgumentError as e:
  # All operations are run during eager execution so an error is raised.
  print(f'{type(e).__name__}: {e}')

In [ ]:
@tf.function
def unused_return_graph(x):
  tf.gather(x, [1]) # unused
  return x

# Only needed operations are run during graph execution. The error is not raised.
print(unused_return_graph(tf.constant([0.0])))

### `tf.function` best practices

可能需要花一些时间来习惯 `Function` 的行为。为了快速上手，初次使用的用户应当使用 `@tf.function` 来装饰简单函数，以获得从 Eager Execution 转换到计算图执行的经验。

*针对 `tf.function`* 进行设计可能是编写与计算图兼容的 TensorFlow 程序的最佳选择。下面是一些提示：

- 尽早并经常使用 `tf.config.run_functions_eagerly` 在 Eager Execution 和计算图执行之间切换，以查明两种模式是否/何时出现分歧。
- Create `tf.Variable`s outside the Python function and modify them on the inside. The same goes for objects that use `tf.Variable`, like `tf.keras.layers`, `tf.keras.Model`s and `tf.keras.optimizers`.
- Avoid writing functions that depend on outer Python variables, excluding `tf.Variable`s and Keras objects. Learn more in *Depending on Python global and free variables* of the [`tf.function` guide](./function.ipynb).
- Prefer to write functions which take tensors and other TensorFlow types as input. You can pass in other object types but be careful! Learn more in *Depending on Python objects* of the [`tf.function` guide](./function.ipynb).
- 在 `tf.function` 下包含尽可能多的计算以最大程度提高性能收益。例如，装饰整个训练步骤或整个训练循环。


## 见证加速

`tf.function` 通常可以提高代码的性能，但加速的程度取决于您运行的计算种类。小型计算可能以调用计算图的开销为主。您可以按如下方式衡量性能上的差异：

In [ ]:
x = tf.random.uniform(shape=[10, 10], minval=-1, maxval=2, dtype=tf.dtypes.int32)

def power(x, y):
  result = tf.eye(10, dtype=tf.dtypes.int32)
  for _ in range(y):
    result = tf.matmul(x, result)
  return result

In [ ]:
print("Eager execution:", timeit.timeit(lambda: power(x, 100), number=1000))

In [ ]:
power_as_graph = tf.function(power)
print("Graph execution:", timeit.timeit(lambda: power_as_graph(x, 100), number=1000))

`tf.function` is commonly used to speed up training loops, and you can learn more about it in the _Speeding-up your training step with `tf.function`_ section of the [Writing a training loop from scratch](https://www.tensorflow.org/guide/keras/writing_a_training_loop_from_scratch) with Keras guide.

Note: You can also try `tf.function(jit_compile=True)` for a more significant performance boost, especially if your code is heavy on TensorFlow control flow and uses many small tensors. Learn more in the _Explicit compilation with `tf.function(jit_compile=True)`_ section of the [XLA overview](https://www.tensorflow.org/xla).

### 性能和权衡

计算图可以加速您的代码，但创建它们的过程有一些开销。对于某些函数，计算图的创建比计算图的执行花费更长的时间。**这种投资通常会随着后续执行的性能提升而迅速得到回报，但重要的是要注意，由于跟踪的原因，任何大型模型训练的前几步可能会较慢。**

No matter how large your model, you want to avoid tracing frequently. The [`tf.function` guide](./function.ipynb) discusses how to set input specifications and use tensor arguments to avoid retracing in the *Controlling retracing* section. If you find you are getting unusually poor performance, it's a good idea to check if you are retracing accidentally.

## `Function` 何时进行跟踪？

要确定您的 `Function` 何时进行跟踪，请在其代码中添加一条 `print` 语句。根据经验，`Function` 将在每次跟踪时执行该 `print` 语句。

In [ ]:
@tf.function
def a_function_with_python_side_effect(x):
  print("Tracing!") # An eager-only side effect.
  return x * x + tf.constant(2)

# This is traced the first time.
print(a_function_with_python_side_effect(tf.constant(2)))
# The second time through, you won't see the side effect.
print(a_function_with_python_side_effect(tf.constant(3)))

In [ ]:
# This retraces each time the Python argument changes,
# as a Python argument could be an epoch count or other
# hyperparameter.
print(a_function_with_python_side_effect(2))
print(a_function_with_python_side_effect(3))

新的 Python 参数总是会触发新计算图的创建，因此需要额外的跟踪。


## 后续步骤

You can learn more about `tf.function` on the API reference page and by following the <a href="./function.ipynb" data-md-type="link">Better performance with `tf.function`</a> guide.